In [1]:
back_end = 'duckdb'
import duckdb
import logging
LOGGER = logging.getLogger(__name__)

In [2]:
import pandas as pd
import pytest

if back_end == 'duckdb':
    import splink.duckdb.duckdb_comparison_level_library as cll
    import splink.duckdb.duckdb_comparison_library as cl
    from splink.duckdb.duckdb_linker import DuckDBLinker
    Linker = DuckDBLinker
elif back_end == 'spark':
    import splink.spark.spark_comparison_level_library as cll
    import splink.spark.spark_comparison_library as cl
    from splink.spark.spark_linker import SparkLinker
    Linker = SparkLinker

In [3]:
if back_end == 'spark':
    from pyspark.context import SparkContext, SparkConf
    from pyspark.sql import SparkSession, types
    from pyspark.sql.functions import col, udf
    from splink.spark.jar_location import similarity_jar_location
    conf = SparkConf()
    path = similarity_jar_location()
    conf.set("spark.jars", path)
    #conf.set("spark.sql.ansi.enabled",True)
    # conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

    sc = SparkContext.getOrCreate(conf=conf)
    spark = SparkSession(sc)
    spark.sparkContext.setCheckpointDir('/Users/alice.oleary/Documents/spark_checkpoint_dir')

    # Register the jaro winkler custom udf
    spark.udf.registerJavaFunction(
        "jaro_winkler", "uk.gov.moj.dash.linkage.JaroWinklerSimilarity", types.DoubleType()
    )

In [4]:
df = pd.DataFrame(
        [
            {
                "unique_id": 1,
                "first_name": "Tom",
                "dob": "02-03-1993",
            },
            {
                "unique_id": 2,
                "first_name": "Robin",
                "dob": "30-01-1992",
            },
        ]
    )


In [5]:
# df = pd.DataFrame(
#         [
#             {
#                 "unique_id": 1,
#                 "first_name": "Tom",
#                 "dob": "2000-13-01",
#             },
#             {
#                 "unique_id": 2,
#                 "first_name": "Robin",
#                 "dob": "2000-01-24",
#             },
#             {
#                 "unique_id": 3,
#                 "first_name": "Zoe",
#                 "dob": "1995-14-30",
#             },
#             {
#                 "unique_id": 4,
#                 "first_name": "Sam",
#                 "dob": "1966-07-05",
#             },
#             {
#                 "unique_id": 5,
#                 "first_name": "Andy",
#                 "dob": "1996-32-15",
#             },
#             {
#                 "unique_id": 6,
#                 "first_name": "Alice",
#                 "dob": "2000-03-25",
#             },
#             {
#                 "unique_id": 7,
#                 "first_name": "Afua",
#                 "dob": "1960-01-01",
#             },
#         ]
#     )


In [6]:
df

,unique_id,first_name,dob
0,1,Tom,02-03-1993
1,2,Robin,30-01-1992


In [7]:
exact_match_fn = cl.exact_match("first_name")

# For testing the cll version
dob_diff = {
    "output_column_name": "dob",
    "comparison_levels": [
        cll.null_level("dob"),
        cll.exact_match_level("dob"),
        cll.datediff_level(
            date_col="dob",
            date_threshold=30,
            date_metric="day",
            cast_strings_to_date=True,
        ),
        cll.datediff_level(
            date_col="dob",
            date_threshold=12,
            date_metric="month",
            cast_strings_to_date=True,
        ),
        cll.datediff_level(
            date_col="dob",
            date_threshold=5,
            date_metric="year",
            cast_strings_to_date=True,
        ),
        cll.datediff_level(
            date_col="dob",
            date_threshold=100,
            date_metric="year",
            cast_strings_to_date=True,
        ),
        cll.else_level(),
    ],
}

In [8]:
settings = {
    "link_type": "dedupe_only",
    "comparisons": [exact_match_fn, dob_diff],
}

settings_cl = {
    "link_type": "dedupe_only",
    "comparisons": [
        exact_match_fn,
        cl.datediff_at_thresholds(
            "dob", [30, 12, 5, 100], ["day", "month", "year", "year"],
            cast_strings_to_date=True
        ),
    ],
}

In [9]:
# df["dob"] = pd.to_datetime(df["dob"])

In [10]:
if back_end == 'spark':
    df_new = spark.createDataFrame(df)
    df_new.persist()

/Users/alice.oleary/Documents/splink_dev_env/lib/python3.10/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/alice.oleary/Documents/splink_dev_env/lib/python3.10/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/alice.oleary/Documents/splink_dev_env/lib/python3.10/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/alice.oleary/Documents/splink_dev_env/lib/python3.10/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. U

In [12]:
with caplog.at_level(logging.WARNING):
    linker = Linker(df_new, settings)
    assert 'Something bad happened!' in caplog.text
# df_e = linker.predict().as_pandas_dataframe()
# linker = Linker(df, settings_cl)
# cl_df_e = linker.predict().as_pandas_dataframe()

NameError: name 'caplog' is not defined

23/04/05 06:03:15 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 36052917 ms exceeds timeout 120000 ms
23/04/05 06:03:16 WARN SparkContext: Killing executors is not supported by current scheduler.


In [ ]:
# settings_cl = {
#     "link_type": "dedupe_only",
#     "comparisons": [
#         exact_match_fn,
#         cl.datediff_at_thresholds(
#             "dob", [30, 12, 5, 100], ["day", "month", "year", "year"],
#             cast_strings_to_date=True, date_format=date_format_param
#         ),
#     ],
# }

In [12]:
def simple_dob_linker(df, dobs=[], date_format_param='', Linker=None):
    settings = {
    "link_type": "dedupe_only",
    "comparisons": [
        exact_match_fn,
        cl.datediff_at_thresholds(
            "dob", [30, 12, 5, 100], ["day", "month", "year", "year"],
            cast_strings_to_date=True, date_format=date_format_param
        ),
    ],
    }
    if len(dobs) == df.shape[0]:
        df['dob'] = dobs
    if back_end == 'spark':
        df = spark.createDataFrame(df)
        df.persist()
    linker = Linker(df, settings)   
    df_e = linker.predict().as_pandas_dataframe()
    return df_e

In [13]:
simple_dob_linker(df, dobs=['03/04/1994', '19/02/1993'], date_format_param='d/m/Y', Linker=Linker)

/Users/alice.oleary/Documents/splink_dev_env/lib/python3.10/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/alice.oleary/Documents/splink_dev_env/lib/python3.10/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/alice.oleary/Documents/splink_dev_env/lib/python3.10/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/alice.oleary/Documents/splink_dev_env/lib/python3.10/site-packages/pyspark/sql/pandas/utils.py:34: DeprecationWarning: distutils Version classes are deprecated. U

KeyboardInterrupt: 

In [ ]:
linker

In [ ]:
# with pytest.raises(duckdb.InvalidInputException):
#     simple_linker(df, settings, Linker)
simple_dob_linker(df, dobs=['03/04/1994', '19/02/1993'], date_format_param='%d/%m/%Y', Linker=Linker)
simple_dob_linker(df, dobs=['03-04-1994', '19-02-1993'], date_format_param='%d-%m-%Y', Linker=Linker)
simple_dob_linker(df, dobs=['04/05/1994', '10/02/1993'], date_format_param='%m/%d/%Y', Linker=Linker)

In [ ]:
if Linker == SparkLinker:
    valid_date_formats = ['dd/mm/Y', 'dd-mm-Y', 'mm/dd/Y', 'Y/mm/dd']
elif Linker == DuckDBLinker:
    valid_date_formats = ['%d/%m/%Y', '%d-%m-%Y', '%m/%d/%Y', '%Y/%m/%d']


In [ ]:
with pytest.raises(py4j.protocol.Py4JJavaError):
    simple_dob_linker(df, dobs=['1994-14-15', '1994-12-03'], 
                        date_format_param='y-M-d', Linker=Linker)

In [ ]:
import py4j

In [ ]:
simple_dob_linker(df, dobs=['1994/14/15', '1994/12/03'], 
                    date_format_param='y/M/d', Linker=Linker)

In [ ]:
# with pytest.raises(duckdb.InvalidInputException):
#     simple_linker(df, settings, Linker)
simple_dob_linker(df, dobs=['03/04/1994', '19/02/1993'], date_format_param='d/m/Y', Linker=Linker)

In [ ]:


simple_dob_linker(df, dobs=['03/04/1994', '19/02/1993'], date_format_param='%d/%m/%Y', Linker=Linker)
simple_dob_linker(df, dobs=['03-04-1994', '19-02-1993'], date_format_param='%d-%m-%Y', Linker=Linker)
simple_dob_linker(df, dobs=['04/05/1994', '10/02/1993'], date_format_param='%m/%d/%Y', Linker=Linker)


In [ ]:
valid_date_formats = ['%d/%m/%Y', '%d-%m-%Y', '%m/%d/%Y']
#valid_date_formats = ['d/m/%Y', 'd-m-%Y', 'm/d/%Y']

In [ ]:
simple_dob_linker(df, dobs=['03/04/1994', '19/02/1993'], date_format_param=valid_date_formats[0], Linker=Linker)

In [ ]:

simple_dob_linker(df, dobs=['03/04/1994', '19/02/1993'], date_format_param='d/m/Y', Linker=Linker)
simple_dob_linker(df, dobs=['03-04-1994', '19-02-1993'], date_format_param='d-m-Y', Linker=Linker)
simple_dob_linker(df, dobs=['04/05/1994', '10/02/1993'], date_format_param='m/d/Y', Linker=Linker)


In [ ]:
simple_dob_linker(df, dobs=['04/05/1994', '10/02/1993'], date_format_param='m/d/Y', Linker=Linker)
simple_dob_linker(df, dobs=['1994/05/04', '1993/14/02'], date_format_param='Y-m-d', Linker=Linker)


In [ ]:
simple_dob_linker(df, dobs=['1994/55/54', '1993/14/02'], date_format_param='%Y-%m-%d', Linker=Linker)


In [ ]:
# incompatible date formats
with pytest.raises(duckdb.InvalidInputException):
    simple_dob_linker(df, dobs=['03-04-1994', '19-02-1993'], date_format_param='%d/%m/%Y', Linker=Linker)


In [ ]:
dobs = ['03-04-1994', '31-02-1993']
df['dob'] = dobs

In [ ]:
settings_cl

In [ ]:
simple_linker(df, settings_cl, Linker)
df_e.head()

In [ ]:
# # Dict key: {size: gamma_level value}
size_gamma_lookup = {1: 11, 2: 6, 3: 3, 4: 1}

linker_outputs = {
    "cll": df_e,
    "cl": cl_df_e,
}

# Check gamma sizes are as expected
for gamma, gamma_lookup in size_gamma_lookup.items():
    for linker_pred in linker_outputs.values():
        assert sum(linker_pred["gamma_dob"] == gamma) == gamma_lookup

# Check individual IDs are assigned to the correct gamma values
# Dict key: {gamma_value: tuple of ID pairs}
size_gamma_lookup = {
    4: [(1, 2)],
    3: [(3, 5), (1, 6), (2, 6)],
    2: [(1, 3), (2, 3), (1, 5), (2, 5), (3, 6), (5, 6)],
}

for gamma, id_pairs in size_gamma_lookup.items():
    for left, right in id_pairs:
        for linker_name, linker_pred in linker_outputs.items():

            print(f"Checking IDs: {left}, {right} for {linker_name}")

            assert (
                linker_pred.loc[
                    (linker_pred.unique_id_l == left)
                    & (linker_pred.unique_id_r == right)
                ]["gamma_dob"].values[0]
                == gamma
            )


In [ ]:
simple_dob_linker(df, dobs=['1994/05/04', '1993/14/02'], date_format_param='%Y/%m/%d', Linker=Linker)

In [ ]:
with pytest.raises(Exception) as e:
    simple_dob_linker(df, dobs=['1994/05/04', '1993/14/02'], date_format_param='%Y/%m/%d', Linker=Linker)
e.type   

In [ ]:
with pytest.raises(Exception) as e:
    simple_dob_linker(df, dobs=['03-14-1994', '19-22-1993'], date_format_param='%d-%m-%Y', Linker=Linker)
e.type   

In [ ]:
with pytest.raises(Exception) as e:
    simple_dob_linker(df, dobs=['20-04-1993', '19-02-1993'], date_format_param='%d/%m/%Y', Linker=Linker)
e.type   

In [ ]:
from splink.duckdb.duckdb_linker import DuckDBLinker
from splink.spark.spark_linker import SparkLinker

In [ ]:
import py4j
if Linker == SparkLinker:
    expected_bad_dates_error = py4j.protocol.Py4JJavaError
elif Linker == DuckDBLinker:
    expected_bad_dates_error = duckdb.InvalidInputException

In [ ]:
with pytest.raises(expected_bad_dates_error):
    simple_dob_linker(df, dobs=['1994/05/04', '1993/14/02'], date_format_param='%Y/%m/%d', Linker=Linker)
with pytest.raises(expected_bad_dates_error):
    simple_dob_linker(df, dobs=['03-14-1994', '19-22-1993'], date_format_param='%d-%m-%Y', Linker=Linker)
with pytest.raises(expected_bad_dates_error):
    simple_dob_linker(df, dobs=['20-04-1993', '19-02-1993'], date_format_param='%d/%m/%Y', Linker=Linker)


In [ ]:
    # # Differing lengths between thresholds and units

cl.datediff_at_thresholds("dob", [1], ["day", "month", "year", "year"])
    # # Negative threshold
    # with pytest.raises(ValueError):
    #     cl.datediff_at_thresholds("dob", [-1], ["day"])
    # # Invalid metric
    # with pytest.raises(ValueError):
    #     cl.datediff_at_thresholds("dob", [1], ["dy"])
    # # Threshold len == 0
    # with pytest.raises(ValueError):
    #     cl.datediff_at_thresholds("dob", [], ["dy"])
    # # Metric len == 0
    # with pytest.raises(ValueError):
    #     cl.datediff_at_thresholds("dob", [1], [])

In [ ]:
cl.datediff_at_thresholds("dob", [1], ["day", "month", "year", "year"])